In [1]:
#Dependencies
import pandas as pd
import numpy as np
import requests
import json
import time

In [2]:
from config import gkey

In [3]:
#Import data, ignore first column in Players.csv

col = range(1,8)
players = pd.read_csv("../Resources/original_data/Players.csv", usecols=col)
player_data = pd.read_csv("../Resources/original_data/player_data.csv")

In [4]:
#Drop unwanted column and rename player column

players.drop(columns=['born', 'weight'], inplace=True)
players.rename(columns={"Player": "name", "collage": "college"}, inplace=True)

In [5]:
#Create seasons_played column using year_start and year_end

player_data['seasons_played'] = player_data['year_end'] - player_data['year_start']

In [6]:
#Drop unwanted columns

player_data.drop(columns=['height', 'weight'], inplace=True)

In [7]:
#Drop rows with no data for birth location

players.dropna(subset=['birth_city', 'birth_state'], inplace=True)

In [8]:
#Merge the two dataframes on player name and college attended

players_merged = pd.merge(players, player_data, on=["name", "college"], how="outer")

In [9]:
#Drop rows with no data for birth location

players_merged.dropna(subset=['birth_city', 'birth_state'], inplace=True)

In [10]:
#Reorder columns

cols=['name','birth_date','birth_city','birth_state','college','height','position','year_start', 'year_end', 'seasons_played']

In [11]:
#New dataframe with ordered columns

players_merged = players_merged[cols]
players_merged

,name,birth_date,birth_city,birth_state,college,height,position,year_start,year_end,seasons_played
0,Cliff Barker,"January 15, 1921",Yorktown,Indiana,University of Kentucky,188.0,G,1950.0,1952.0,2.0
1,Ralph Beard,"December 2, 1927",Hardinsburg,Kentucky,University of Kentucky,178.0,G,1950.0,1951.0,1.0
2,Charlie Black,"June 15, 1921",Arco,Idaho,University of Kansas,196.0,F-C,1949.0,1952.0,3.0
3,Nelson Bobb,"February 25, 1924",Philadelphia,Pennsylvania,Temple University,183.0,G,1950.0,1953.0,3.0
4,Jake Bornheimer,"June 29, 1927",New Brunswick,New Jersey,Muhlenberg College,196.0,F-C,1949.0,1950.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3437,Troy Williams,NaN,Columbia,South Carolina,South Carolina State University,198.0,NaN,NaN,NaN,NaN
3438,Kyle Wiltjer,"October 20, 1992",Portland,Oregon,Gonzaga University,208.0,F,2017.0,2017.0,0.0
3439,Stephen Zimmerman,"September 9, 1996",Hendersonville,Tennessee,"University of Nevada, Las Vegas",213.0,C,2017.0,2017.0,0.0
3440,Paul Zipser,"February 18, 1994",Heidelberg,Germany,NaN,203.0,G-F,2017.0,2018.0,1.0


In [12]:
#List of US states

US_states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","District of Columbia","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

In [13]:
#Filter for all players in the US

us_df = players_merged[players_merged['birth_state'].isin(US_states)]
us_df

,name,birth_date,birth_city,birth_state,college,height,position,year_start,year_end,seasons_played
0,Cliff Barker,"January 15, 1921",Yorktown,Indiana,University of Kentucky,188.0,G,1950.0,1952.0,2.0
1,Ralph Beard,"December 2, 1927",Hardinsburg,Kentucky,University of Kentucky,178.0,G,1950.0,1951.0,1.0
2,Charlie Black,"June 15, 1921",Arco,Idaho,University of Kansas,196.0,F-C,1949.0,1952.0,3.0
3,Nelson Bobb,"February 25, 1924",Philadelphia,Pennsylvania,Temple University,183.0,G,1950.0,1953.0,3.0
4,Jake Bornheimer,"June 29, 1927",New Brunswick,New Jersey,Muhlenberg College,196.0,F-C,1949.0,1950.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3435,Okaro White,"August 13, 1992",Clearwater,Florida,Florida State University,203.0,F,2017.0,2018.0,1.0
3436,Isaiah Whitehead,"March 8, 1995",Brooklyn,New York,Seton Hall University,193.0,G,2017.0,2018.0,1.0
3437,Troy Williams,NaN,Columbia,South Carolina,South Carolina State University,198.0,NaN,NaN,NaN,NaN
3438,Kyle Wiltjer,"October 20, 1992",Portland,Oregon,Gonzaga University,208.0,F,2017.0,2017.0,0.0


In [14]:
#Top US cities

us_df['birth_city'].value_counts().head(10)

Chicago         114
Los Angeles      88
Philadelphia     83
Brooklyn         76
New York         71
Washington       62
Detroit          58
Atlanta          37
Baltimore        37
Dallas           33
Name: birth_city, dtype: int64

In [15]:
#Top US states

us_df['birth_state'].value_counts().head(10)

California        345
New York          290
Illinois          209
Pennsylvania      163
Ohio              137
Michigan          131
Texas             129
Georgia           118
Louisiana         101
North Carolina     96
Name: birth_state, dtype: int64

In [16]:
#Filter for all players outside of the US

international_df = players_merged[~players_merged['birth_state'].isin(US_states)]
international_df

,name,birth_date,birth_city,birth_state,college,height,position,year_start,year_end,seasons_played
64,Lee Knorek,"July 15, 1921",Warsaw,Poland,University of Detroit Mercy,201.0,C,1947.0,1950.0,3.0
131,Ernie Vandeweghe,"September 12, 1928",Montreal,Canada,Colgate University,190.0,F-G,1950.0,1956.0,6.0
184,Bob Houbregs*,NaN,Vancouver,Canada,University of Washington,201.0,NaN,NaN,NaN,NaN
217,Chris Harris,"August 11, 1933",Southampton,United Kingdom,University of Dayton,190.0,G,1956.0,1956.0,0.0
314,Tom Meschery,"October 26, 1938",Harbin Manchuria,China,Saint Mary's College of California,198.0,F,1962.0,1971.0,9.0
...,...,...,...,...,...,...,...,...,...,...
3424,Dario Saric,"April 8, 1994",Sibenik,Croatia,NaN,208.0,F,2017.0,2018.0,1.0
3425,Tomas Satoransky,"October 30, 1991",Prague,Czech Republic,NaN,201.0,G,2017.0,2018.0,1.0
3427,Pascal Siakam,"February 4, 1994",Douala,Cameroon,New Mexico State University,206.0,F,2017.0,2018.0,1.0
3440,Paul Zipser,"February 18, 1994",Heidelberg,Germany,NaN,203.0,G-F,2017.0,2018.0,1.0


In [17]:
#Top international countries

international_df['birth_state'].value_counts().head(20)

Canada                    26
Serbia                    20
France                    19
Germany                   17
Croatia                   16
Brazil                    15
Spain                     14
Australia                 13
Argentina                 12
Lithuania                 11
Bosnia and Herzegovina    11
Russia                    10
Slovenia                   9
Nigeria                    9
United Kingdom             9
Greece                     8
Senegal                    8
Ukraine                    8
Jamaica                    7
Italy                      7
Name: birth_state, dtype: int64

In [18]:
#Top international cities

international_df['birth_city'].value_counts().head(20)

Toronto          10
Belgrade          9
Kaunas            7
Melbourne         7
Split             7
Dakar             6
Montreal          6
Paris             5
Madrid            5
Kingston          5
London            5
Ljubljana         4
Nassau            3
Mostar            3
Istanbul          3
Barcelona         3
Kinshasa          3
Santo Domingo     3
Santa Fe          3
Lagos             3
Name: birth_city, dtype: int64

In [19]:
coordinates_df = pd.read_csv("../Resources/clean_data/US_players_coordinates.csv")

In [20]:
coordinates_df.rename(columns={"Birth City": "birth_city",
                               "Birth State": "birth_state",
                               "Year Start": "year_start"}, inplace=True)

In [21]:
player_data = pd.merge(players_merged, coordinates_df, on=['birth_city', 'birth_state', 'year_start'], how="outer")

In [23]:
player_data.drop(columns=['Unnamed: 0'], inplace=True)

In [24]:
player_data

,name,birth_date,birth_city,birth_state,college,height,position,year_start,year_end,seasons_played,Latitude (generated),Longitude (generated)
0,Cliff Barker,"January 15, 1921",Yorktown,Indiana,University of Kentucky,188.0,G,1950.0,1952.0,2.0,40.1737,-85.4941
1,Jack Parkinson,"March 4, 1924",Yorktown,Indiana,University of Kentucky,183.0,G,1950.0,1950.0,0.0,40.1737,-85.4941
2,Ralph Beard,"December 2, 1927",Hardinsburg,Kentucky,University of Kentucky,178.0,G,1950.0,1951.0,1.0,37.7801,-86.4605
3,Charlie Black,"June 15, 1921",Arco,Idaho,University of Kansas,196.0,F-C,1949.0,1952.0,3.0,43.6353,-113.3000
4,Nelson Bobb,"February 25, 1924",Philadelphia,Pennsylvania,Temple University,183.0,G,1950.0,1953.0,3.0,40.0115,-75.1327
...,...,...,...,...,...,...,...,...,...,...,...,...
3437,Okaro White,"August 13, 1992",Clearwater,Florida,Florida State University,203.0,F,2017.0,2018.0,1.0,27.9659,-82.8001
3438,Isaiah Whitehead,"March 8, 1995",Brooklyn,New York,Seton Hall University,193.0,G,2017.0,2018.0,1.0,40.6922,-73.9902
3439,Stephen Zimmerman,"September 9, 1996",Hendersonville,Tennessee,"University of Nevada, Las Vegas",213.0,C,2017.0,2017.0,0.0,36.3048,-86.6200
3440,Paul Zipser,"February 18, 1994",Heidelberg,Germany,NaN,203.0,G-F,2017.0,2018.0,1.0,NaN,NaN


In [25]:
geo_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
geo_params = {
    'address':'Austin, Texas',
    'key':gkey
}

In [28]:
for index, row in player_data.iterrows():
    if row['Latitude (generated)'].isnull():
        print(row['name'])

AttributeError: 'float' object has no attribute 'isnull'

In [26]:
for index, row in player_data.iterrows():
    if row['Latitude (generated)'].isnull():
        geo_params['address'] = f"{row['birth_city']}, {row['birth_state']}"
        geo_response = requests.get(geo_url, geo_params).json()
        players_merged.at[index, 'Latitude (generated)'] = geo_response["results"][0]["geometry"]["location"]["lat"]
        players_merged.at[index, 'Longitude (generated)'] = geo_response["results"][0]["geometry"]["location"]["lng"]

KeyboardInterrupt: 

In [ ]:
players_merged

In [ ]:
#Export US players as csv

us_df.to_csv('../Resources/clean_data/US_players.csv')

In [ ]:
#Export international players as csv

international_df.to_csv('../Resources/clean_data/International_players.csv')

In [ ]:
#Export entire player dataframe as csv

players_merged.to_csv('../Resources/clean_data/All_players.csv', index=True)